In [69]:
#To check for precedents
import json
with open('jsonformatter.txt', 'r') as f:
    with open("Precedents output.txt", 'w') as out:
        data = json.load(f)

        # print(type(data['label']))
        for case in data:
            out.write('Case id : ' + str(case['id']) + '\n')
            annotation = case['annotations'][0]
            # print(annotation['result'])
            for key in annotation['result']:
                # out.write(key['value']['labels'][0] + '\n')
                if key['value']['labels'][0] == 'STA':
                    out.write('STA : ' + key['value']['text'] + '\n\n')

            out.write("\n")

In [3]:
#To check for precedents relied
import json
with open('jsonformatter.txt', 'r') as f:
    with open("Precedents relied output.txt", 'w') as out:
        data = json.load(f)

        for case in data:
            out.write('Id : ' + str(case['id']) + '\n')
            annotation = case['annotations'][0]
            for key in annotation['result']:
                if key['value']['labels'][0] == 'PRE_RELIED':
                    out.write('PRE_RELIED : ' + key['value']['text'] + '\n\n')

            out.write("\n")

In [4]:
#To check for precedents not relied
import json
with open('jsonformatter.txt', 'r') as f:
    with open("Precedents not relied output.txt", 'w') as out:
        data = json.load(f)

        for case in data:
            out.write('Id : ' + str(case['id']) + '\n')
            annotation = case['annotations'][0]
            for key in annotation['result']:
                if key['value']['labels'][0] == 'PRE_NOT_RELIED':
                    out.write('PRE_NOT_RELIED : ' + key['value']['text'] + '\n\n')

            out.write("\n")

In [8]:
#To seperate the precedents on the keyword "vs" or "v."

import xml.etree.ElementTree as ET
import os

directories = ['Writ Petition', 'Civil Appeal']
for directory in directories:
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        mytree = ET.ElementTree(file= f)
        myroot = mytree.getroot()

        out_dir = directory+' precedents output'
        out_file = os.path.join(out_dir, filename.replace(".xml", " output.txt"))
        with open( out_file, 'w') as out:
            for i in myroot:
                if(i.tag=='JudgmentText'):
                    for legis in i:
                        # out.write("Inside Judgement Text : " + legis.tag + '\n')
                        if(legis.tag=='I'):
                            for para in legis:
                                if para.text is None:
                                    continue
                                s1 = para.text.lower()
                                s2 = para.text
                                if ' vs' in s1 or ' v. ' in s2 or ' v ' in s2:
                                    out.write("Precedent : " + para.text + '\n\n')

                        elif(legis.tag=='P'):

                            if legis.text is None:
                                continue

                            s1 = legis.text.lower()
                            s2 = legis.text
                            if ' vs' in s1 or ' v. ' in s2 or ' v ' in s2:
                                out.write("Precedent : " + legis.text + '\n\n')

In [14]:
#To classify precedents

import xml.etree.ElementTree as ET
import os
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()
directories = ['Writ Petition', 'Civil Appeal']
for directory in directories:
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        mytree = ET.ElementTree(file= f)
        myroot = mytree.getroot()

        out_dir = directory+' precedents classified output'
        out_file = os.path.join(out_dir, filename.replace(".xml", " output.txt"))
        with open( out_file, 'w') as out:
            for i in myroot:
                if(i.tag=='JudgmentText'):
                    for legis in i:
                        if(legis.tag=='I'):
                            for para in legis:
                                if para.text is None:
                                    continue
                                s1 = para.text.lower()
                                s2 = para.text
                                if ' vs' in s1 or ' v. ' in s2 or ' v ' in s2:
                                    if 'overruled' in s1:
                                        out.write("Precedent Overruled : ")
                                    elif 'distinguish' in s1:
                                        out.write("Precedent Distinguished : ")
                                    elif sia.polarity_scores(s2)["compound"] > 0:
                                        out.write("Precedent Relied : ")
                                    else:
                                        out.write("Precedent Referred : ")
                                    out.write(s2 + '\n\n')

                        elif(legis.tag=='P'):

                            if legis.text is None:
                                continue

                            s1 = legis.text.lower()
                            s2 = legis.text
                            if ' vs' in s1 or ' v. ' in s2 or ' v ' in s2:
                                if 'overruled' in s1:
                                    out.write("Precedent Overruled : ")
                                elif 'distinguish' in s1:
                                    out.write("Precedent Distinguished : ")
                                elif sia.polarity_scores(s2)["compound"] > 0:
                                    out.write("Precedent Relied : ")
                                else:
                                    out.write("Precedent Referred : ")
                                out.write(s2 + '\n\n')

In [15]:
#Sentence wise classification of precedents

import xml.etree.ElementTree as ET
import os
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize

sia = SentimentIntensityAnalyzer()
directories = ['Writ Petition', 'Civil Appeal']
for directory in directories:
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        mytree = ET.ElementTree(file= f)
        myroot = mytree.getroot()

        out_dir = directory+' precedents classified output'
        out_file = os.path.join(out_dir, filename.replace(".xml", " output.txt"))
        with open( out_file, 'w') as out:
            for i in myroot:
                if(i.tag=='JudgmentText'):
                    for legis in i:
                        if(legis.tag=='I'):
                            for para in legis:
                                if para.text is None:
                                    continue
                                flag = 0
                                s = ""
                                for sent in sent_tokenize(para.text):
                                    s1 = sent.lower()
                                    s2 = sent
                                    if ' vs' in s1 or ' v. ' in s2 or ' v ' in s2:
                                        flag=1
                                        s += s2
                                if flag:
                                    if 'overruled' in s.lower():
                                        out.write("Precedent Overruled : ")
                                    elif 'distinguish' in s.lower():
                                        out.write("Precedent Distinguished : ")
                                    elif sia.polarity_scores(s2)["compound"] > 0:
                                        out.write("Precedent Relied : ")
                                    else:
                                        out.write("Precedent Referred : ")
                                    out.write(s + '\n\n')

                        elif(legis.tag=='P'):

                            if legis.text is None:
                                continue

                            flag = 0
                            s = ""
                            for sent in sent_tokenize(legis.text):
                                s1 = sent.lower()
                                s2 = sent
                                if ' vs' in s1 or ' v. ' in s2 or ' v ' in s2:
                                    flag=1
                                    s += s2
                            if flag:
                                if 'overruled' in s.lower():
                                    out.write("Precedent Overruled : ")
                                elif 'distinguish' in s.lower():
                                    out.write("Precedent Distinguished : ")
                                elif sia.polarity_scores(s2)["compound"] > 0:
                                    out.write("Precedent Relied : ")
                                else:
                                    out.write("Precedent Referred : ")
                                out.write(s + '\n\n')

In [28]:
#To differentiate between capital V and small v

import xml.etree.ElementTree as ET
import os

with open( 'Precedents output.txt', 'w') as out:
    directories = ['Writ Petition', 'Civil Appeal']
    for directory in directories:
        for filename in os.listdir(directory):
            f = os.path.join(directory, filename)
            mytree = ET.ElementTree(file= f)
            myroot = mytree.getroot()

            for i in myroot:
                if(i.tag=='JudgmentText'):
                    for legis in i:
                        # out.write("Inside Judgement Text : " + legis.tag + '\n')
                        if(legis.tag=='I'):
                            for para in legis:
                                if para.text is None:
                                    continue
                                s1 = para.text.lower()
                                s2 = para.text
                                if 'overrule' in s1: # or (' v ' in s1 and ' v ' not in s2): 
                                    out.write(directory + ' : ' + filename  + ' : ' + "Precedent : " + para.text + '\n\n')

                        elif(legis.tag=='P'):

                            if legis.text is None:
                                continue

                            
                            s1 = legis.text.lower()
                            s2 = legis.text
                            if 'overrule' in s1: # or (' v ' in s1 and ' v ' not in s2): 
                                out.write(directory + ' : '  + filename + ' : '  + "Precedent : " + legis.text + '\n\n')

In [4]:
#Sentence wise classification of precedents in pdf format

import PyPDF2
import os
# import nltk.data
from nltk.tokenize import sent_tokenize

# tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
directories = ['New cases']
for directory in directories:
    out_dir = directory+' precedents classified output'
    for filename in os.listdir(directory):
        # f = os.path.join(directory, filename)
        out_file = os.path.join(out_dir, filename.replace(".pdf", " output.txt"))
        out_file.encode('utf-8')
        with open("{}/{}".format(directory,filename), 'r', encoding = 'utf-8') as pdfFileObj:
            object = PyPDF2.PdfFileReader(pdfFileObj)
            NumPages = object.getNumPages()
            with open( out_file, 'w') as out:
                for i in range(NumPages):
                    Text = object.getPage(i).extractText()
                    flag = 0
                    s = ""
                    for sent in sent_tokenize(Text):
                        s1 = sent.lower()
                        s2 = sent
                        if ' vs' in s1 or ' v. ' in s2 or ' v ' in s2:
                            flag=1
                            s += s2
                    if flag:
                        if 'overruled' in s.lower():
                            out.write("Precedent Overruled : ")
                        elif 'distinguish' in s.lower():
                            out.write("Precedent Distinguished : ")
                        elif sia.polarity_scores(s2)["compound"] > 0:
                            out.write("Precedent Relied : ")
                        else:
                            out.write("Precedent Referred : ")
                        out.write(s + '\n\n')

PdfReader stream/file object is not in binary mode. It may not be read correctly.


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xad in position 0: invalid start byte

In [20]:
#Sentence wise classification of precedents

import xml.etree.ElementTree as ET
import os
# import nltk.data
from nltk.tokenize import sent_tokenize

# tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
directories = ['Annotated Data_with xml and original PDF']
for directory in directories:
    for filename in os.listdir(directory):
        if '.xml' in filename:
            f = os.path.join(directory, filename)
            mytree = ET.parse(f)
            # myroot = mytree.getroot()

            out_dir = directory+' precedents classified output'
            out_file = os.path.join(out_dir, filename.replace(".xml", " output.txt"))
            with open( out_file, 'w') as out:
                for i in mytree.iter():
                    if i.text is not None:
                        out.write(i.text + '\n')
                    

In [17]:
#Sentence wise classification of precedents from txt file for 26 cases

import os
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

directories = ['Twenty_six']
for directory in directories:
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        out_dir = directory+' output'
        out_file = os.path.join(out_dir, filename.replace(".txt", " output.txt"))
        with open(f, 'r') as in_file:
            with open( out_file, 'w') as out:
                for s2 in in_file:
                    # out.write(line + '\n')
                    flag = 0
                    s1 = s2.lower()
                    if ' vs' in s1 or ' v. ' in s2 or ' v ' in s2:
                        flag=1
                    if flag:
                        if 'overruled' in s1.lower():
                            out.write("Precedent Overruled : ")
                        elif 'distinguish' in s1.lower():
                            out.write("Precedent Distinguished : ")
                        elif sia.polarity_scores(s2)["compound"] > 0:
                            out.write("Precedent Relied : ")
                        else:
                            out.write("Precedent Referred : ")
                        out.write(s1 + '\n')

In [37]:
#Code to extract and classify precedents from any XML file with text within JudgementText
import os
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize

sia = SentimentIntensityAnalyzer()

directories = ['Annotated Data_with xml and original PDF']
for directory in directories:
    out_dir = directory+' output'
    for filename in os.listdir(directory):
        if '.xml' in filename:
            f = os.path.join(directory, filename)
            out_file = os.path.join(out_dir, filename.replace(".xml", " output.txt"))
            with open(f, 'r') as file:
                with open( out_file, 'w') as out:
                    plain_text = ""
                    for line in file:
                        plain_text += line
                    sub1 = '<JudgmentText>'
                    sub2 = '</JudgmentText>'
                    idx1 = plain_text.index(sub1) + len(sub1)
                    idx2 = plain_text.index(sub2)
                    text = ""
                    flag=1
                    for i in range(idx1, idx2):
                        char = plain_text[i]
                        if char == '<':
                            flag=0
                        elif char == '>':
                            flag=1
                        elif flag:
                            # if char != '\n':# or text[-1] == '.':
                            text += char
                    text = " ".join(text.split())
                    # out.write(text + '\n\n')
                    # sentences = sent_tokenize(text)
                    for sent in sent_tokenize(text):
                        # out.write(sent + '\n\n')
                        s1 = sent.lower()
                        s2 = sent
                        if ' vs' in s1 or ' v. ' in s2 or ' v ' in s2:
                            if 'overruled' in s1:
                                out.write("Precedent Overruled : ")
                            elif 'distinguish' in s1:
                                out.write("Precedent Distinguished : ")
                            elif sia.polarity_scores(s2)["compound"] > 0:
                                out.write("Precedent Relied : ")
                            else:
                                out.write("Precedent Referred : ")
                            out.write(sent + '\n\n')